In [14]:
# Imports and loading data
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

df = pd.read_csv("steel.csv")
df.head()

,normalising_temperature,tempering_temperature,percent_silicon,percent_chromium,percent_copper,percent_nickel,percent_sulphur,percent_carbon,percent_manganese,tensile_strength
0,178.500,275,0.1530,0.970575,0.942,0.8870,0.0,1.920,0.0,25.107613
1,178.500,950,0.1530,1.212726,0.942,0.8870,0.0,1.920,0.0,140.035334
2,178.500,375,0.1530,1.621165,0.942,0.8870,0.0,1.920,0.0,42.217650
3,178.500,900,0.1530,0.809989,0.942,0.8870,0.0,1.920,0.0,95.015309
4,189.525,900,0.1624,1.036229,0.849,0.9382,0.0,2.035,0.0,113.266773


In [15]:
# Define features and target vairables
target_col = 'tensile_strength'
X = df.drop(columns=[target_col]).values
y = df[target_col].values
feature_names = df.drop(columns=[target_col]).columns.tolist()

print("Features:", feature_names)
print("Target:", target_col)

Features: ['normalising_temperature', 'tempering_temperature', 'percent_silicon', 'percent_chromium', 'percent_copper', 'percent_nickel', 'percent_sulphur', 'percent_carbon', 'percent_manganese']
Target: tensile_strength


In [16]:
# metrics to compare performance
# RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
# MAPE
def mape(y_true, y_pred):
    eps = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + eps))) * 100

In [17]:
# Experimental setup
RANDOM_STATE = 42
n_splits = 10
outer_cv = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)

In [ ]:
# Default RFR
rf_default = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=RANDOM_STATE,
    n_jobs=1
)

results_default = []

for train_idx, test_idx in outer_cv.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Scaling features  
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    # Train default RFR
    rf_default.fit(X_train_s, y_train)

    # Predictions
    y_train_pred = rf_default.predict(X_train_s)
    y_test_pred = rf_default.predict(X_test_s)

    # Check RMSE and MAPE
    results_default.append({
        'train_rmse': rmse(y_train, y_train_pred),
        'test_rmse': rmse(y_test, y_test_pred),
        'train_mape': mape(y_train, y_train_pred),
        'test_mape': mape(y_test, y_test_pred)
    })

# Summary table for default RFR
df_rf_default = pd.DataFrame(results_default)
display(df_rf_default)

# %matplotlib inline needed for display not working
summary_rf_default = df_rf_default.agg(['mean', 'std']).T
print("\nDefault Random Forest Averages:")
display(summary_rf_default)

,train_rmse,test_rmse,train_mape,test_mape
0,11.142486,29.469903,5.494718,18.474158
1,11.345121,27.183563,5.319342,12.984641
2,11.110018,26.030787,5.214582,12.405952
3,11.315080,24.165296,5.457373,9.696113
4,10.583881,32.876818,5.097921,14.785660
5,10.850718,22.694676,5.126409,12.191052
6,11.170730,31.838054,5.317354,16.523374
7,10.634581,30.976195,4.996063,15.464545
8,10.383807,34.099235,5.153949,16.334586
9,10.876687,23.545794,5.066969,11.838133



Default Random Forest Averages:


,mean,std
train_rmse,10.941311,0.328188
test_rmse,28.288032,4.124980
train_mape,5.224468,0.167526
test_mape,14.069821,2.678942
